In [ ]:
import pandas as pd
import numpy as np

from IPython.display import display

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Para calcular las metricas de los modelos se va a utilizar un dataset que consta de 4800 criticas cinematograficas, dicho dataset fue bajado en el siguiente link

https://raw.githubusercontent.com/captain500/elmundodelosdatos/main/5_sentiment_analysis/data/criticas.csv

Que es un data set que proporsiona la pagina *elmundodelosdatos* en el siguiente link (https://elmundodelosdatos.com/sentiment-analysis-criticas-peliculas-regresion-logistica/) que toma criticas cinematograficas de la pagina filmaffinity (www.filmaffinity.com).

Las criticas estan clasificadas con una nota de 1 a 10, se tomo la hipotesis que aquellas peliculas con un nota mayor o igual a 7 es una critica positiva, de lo contrario es una critica negativa.

En esta notebook se hara la adaptacion y procesamiento correspondiente al dataset para ser utilizado en el presente trabajo para posteriormente calcular las metricas de los modelos usando este dataset

#Preprocesamiento del dataset para hallar las metricas

In [ ]:
path = 'https://raw.githubusercontent.com/captain500/elmundodelosdatos/main/5_sentiment_analysis/data/criticas.csv'
df_train = pd.read_csv(path)

In [ ]:
df_train

,critica,nota,url
0,"Bueno, bajo mi gusto, otro fracaso más de DC. ...",3,https://www.filmaffinity.com/es/reviews/1/4208...
1,Es tan terrible que podría funcionar como paro...,1,https://www.filmaffinity.com/es/reviews/1/4208...
2,Tengo una tradición desde hace más de 5 años. ...,2,https://www.filmaffinity.com/es/reviews/1/4208...
3,No entiendo como nadie tiene la cara de presen...,1,https://www.filmaffinity.com/es/reviews/1/4208...
4,La primera entrega de Wonder Woman (2017) no m...,4,https://www.filmaffinity.com/es/reviews/1/4208...
...,...,...,...
4795,"""Shrek"" es sin lugar a dudas una de las mejore...",9,https://www.filmaffinity.com/es/reviews/6/4945...
4796,"Muy buena e incluso diría, inteligente comedia...",8,https://www.filmaffinity.com/es/reviews/3/9420...
4797,Cuando una película consigue hacer que algo ta...,7,https://www.filmaffinity.com/es/reviews/3/9420...
4798,Una gran comedia estupida que cumple su funció...,8,https://www.filmaffinity.com/es/reviews/3/9420...


Cambiar los nombres de las columnas

In [ ]:
df_train = df_train.rename(columns={'critica': 'review_es'})
df_train = df_train.rename(columns={'nota': 'sentimiento'})
df_train

,review_es,sentimiento,url
0,"Bueno, bajo mi gusto, otro fracaso más de DC. ...",3,https://www.filmaffinity.com/es/reviews/1/4208...
1,Es tan terrible que podría funcionar como paro...,1,https://www.filmaffinity.com/es/reviews/1/4208...
2,Tengo una tradición desde hace más de 5 años. ...,2,https://www.filmaffinity.com/es/reviews/1/4208...
3,No entiendo como nadie tiene la cara de presen...,1,https://www.filmaffinity.com/es/reviews/1/4208...
4,La primera entrega de Wonder Woman (2017) no m...,4,https://www.filmaffinity.com/es/reviews/1/4208...
...,...,...,...
4795,"""Shrek"" es sin lugar a dudas una de las mejore...",9,https://www.filmaffinity.com/es/reviews/6/4945...
4796,"Muy buena e incluso diría, inteligente comedia...",8,https://www.filmaffinity.com/es/reviews/3/9420...
4797,Cuando una película consigue hacer que algo ta...,7,https://www.filmaffinity.com/es/reviews/3/9420...
4798,Una gran comedia estupida que cumple su funció...,8,https://www.filmaffinity.com/es/reviews/3/9420...


Eliminar la columna que no es importarte para el preprocesamiento.

In [ ]:
df_train = df_train.drop(columns=['url'])

Colocar los sentimientos de cada critica.

0 ---> positiva

1 ---> negativa

In [ ]:
df_train['sentimiento'] = df_train['sentimiento'].apply(lambda x: 0 if x > 6 else 1)

In [ ]:
df_train

,review_es,sentimiento
0,"Bueno, bajo mi gusto, otro fracaso más de DC. ...",1
1,Es tan terrible que podría funcionar como paro...,1
2,Tengo una tradición desde hace más de 5 años. ...,1
3,No entiendo como nadie tiene la cara de presen...,1
4,La primera entrega de Wonder Woman (2017) no m...,1
...,...,...
4795,"""Shrek"" es sin lugar a dudas una de las mejore...",0
4796,"Muy buena e incluso diría, inteligente comedia...",0
4797,Cuando una película consigue hacer que algo ta...,0
4798,Una gran comedia estupida que cumple su funció...,0


##Limpieza del texto

In [ ]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 44.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.8 MB/s eta 0:00:00


In [ ]:
from unidecode import unidecode
import re
import spacy

In [ ]:
def limpiar(texto):
  texto = texto.lower() #todo a minusculas
  texto = unidecode(texto) #eliminar acentos
  texto = re.sub(r'[^a-zA-Z\s]', '', texto)#Elimina todos los caracteres que no sean letras (de la A a la Z, sin importar mayúsculas o minúsculas) o espacios.
  texto = re.sub(r'\W', ' ', str(texto))#Reemplaza cualquier carácter no alfanumérico con un espacio.
  texto = re.sub(r'\s+[a-zA-Z]\s+', ' ', texto)#Elimina palabras que consisten en un solo carácter.
  texto = re.sub(r'\s+', ' ', texto, flags=re.I)#Reemplaza múltiples espacios en blanco consecutivos con un solo espacio
  return texto

In [ ]:
df_train['review_es'] = df_train['review_es'].apply(limpiar)

In [ ]:
df_train.to_csv('/content/drive/MyDrive/DataSet prueba/df_prueba_limpiado.csv')

In [ ]:
df_train

,review_es,sentimiento
0,bueno bajo mi gusto otro fracaso mas de dc emp...,1
1,es tan terrible que podria funcionar como paro...,1
2,tengo una tradicion desde hace mas de anos en ...,1
3,no entiendo como nadie tiene la cara de presen...,1
4,la primera entrega de wonder woman no me parec...,1
...,...,...
4795,shrek es sin lugar dudas una de las mejores pe...,0
4796,muy buena incluso diria inteligente comedia pa...,0
4797,cuando una pelicula consigue hacer que algo ta...,0
4798,una gran comedia estupida que cumple su funcio...,0


##Insertar stop words

In [ ]:
!pip install stop_words
import stop_words
# Usamos stop words del español
stop_words = stop_words.get_stop_words('es')

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=16c9ccc02e0b716f504e318adfe413995c634f9540d42cca5d326a966cb692d6
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [ ]:
display(stop_words)

['a',
 'al',
 'algo',
 'algunas',
 'algunos',
 'ante',
 'antes',
 'como',
 'con',
 'contra',
 'cual',
 'cuando',
 'de',
 'del',
 'desde',
 'donde',
 'durante',
 'e',
 'el',
 'ella',
 'ellas',
 'ellos',
 'en',
 'entre',
 'era',
 'erais',
 'eran',
 'eras',
 'eres',
 'es',
 'esa',
 'esas',
 'ese',
 'eso',
 'esos',
 'esta',
 'estaba',
 'estabais',
 'estaban',
 'estabas',
 'estad',
 'estada',
 'estadas',
 'estado',
 'estados',
 'estamos',
 'estando',
 'estar',
 'estaremos',
 'estará',
 'estarán',
 'estarás',
 'estaré',
 'estaréis',
 'estaría',
 'estaríais',
 'estaríamos',
 'estarían',
 'estarías',
 'estas',
 'este',
 'estemos',
 'esto',
 'estos',
 'estoy',
 'estuve',
 'estuviera',
 'estuvierais',
 'estuvieran',
 'estuvieras',
 'estuvieron',
 'estuviese',
 'estuvieseis',
 'estuviesen',
 'estuvieses',
 'estuvimos',
 'estuviste',
 'estuvisteis',
 'estuviéramos',
 'estuviésemos',
 'estuvo',
 'está',
 'estábamos',
 'estáis',
 'están',
 'estás',
 'esté',
 'estéis',
 'estén',
 'estés',
 'fue',
 'f

Eliminar stop words del texto

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Pre: todas las palabras deben de estar en minuscula
def eliminar_palabras(text):
    texto_tokenizado = word_tokenize(text) #Tokenizar texto
    texto_filtrado = [palabra for palabra in texto_tokenizado if palabra not in stop_words]
    return ' '.join(texto_filtrado)

In [ ]:
df_train['review_es'] = df_train['review_es'].apply(eliminar_palabras)

In [ ]:
df_train.to_csv('/content/drive/MyDrive/DataSet prueba/df_prueba_limpiado_stopWords.csv')

In [ ]:
df_train

,review_es,sentimiento
0,bueno bajo gusto fracaso mas dc empezo ano ave...,1
1,tan terrible podria funcionar parodia tan sobr...,1
2,tradicion hace mas anos cada diciembre voy mis...,1
3,entiendo nadie cara presentar pelicula entiend...,1
4,primera entrega wonder woman parecio ninguna m...,1
...,...,...
4795,shrek lugar dudas mejores peliculas animacion ...,0
4796,buena incluso diria inteligente comedia sorpre...,0
4797,pelicula consigue hacer tan normal salir calle...,0
4798,gran comedia estupida cumple funcion perfeccio...,0


##Lematizar texto

In [ ]:
nlp = spacy.load('es_core_news_sm')

In [ ]:
def lematizar(texto):
    doc = nlp(texto)#pasa el texto por el modelo de spaCy, devolviendo un objeto doc que contiene una lista de tokens.
    lemas = [token.lemma_ for token in doc]#Extrae el lema de cada token (palabra) en cada doc
    return ' '.join(lemas)#une todos los Lemas del doc en un Solo Texto separados por un espaio

In [ ]:
df_train['review_es'] = df_train['review_es'].apply(lematizar)

In [ ]:
df_train.to_csv('/content/drive/MyDrive/DataSet prueba/df_prueba_preprocesado.csv')

In [ ]:
df_train

,review_es,sentimiento
0,bueno bajo gusto fracaso mas dc empezo ano ave...,1
1,tanto terrible podrio funcionar parodia tanto ...,1
2,tradicion hacer mas ano cada diciembre ir mism...,1
3,entender nadie cara presentar pelicula entende...,1
4,primero entrega wonder woman parecio ninguno m...,1
...,...,...
4795,shrek lugar duda mejor pelicula animacion pode...,0
4796,buen incluso dirio inteligente comedio sorpres...,0
4797,pelicula conseguir hacer tanto normal salir ca...,0
4798,gran comedia estupido cumplir funcion perfecci...,0
